<a href="https://colab.research.google.com/github/Astenik/svd_applications/blob/main/movie_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy==1.24.4 scipy==1.10.1
!pip install scikit-surprise --no-binary scikit-surprise

  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2469550 sha256=a3cd3b7cbba9e4048093fdd7234cda1525e44108005426672c1f73a7d4bbc687
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [ ]:
from google.colab import files
import requests
from surprise.model_selection import cross_validate
import json
import os
from surprise import Dataset, Reader, SVD, accuracy
import pandas as pd
from surprise.model_selection import train_test_split
import random
import time

!wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip

--2025-07-29 14:11:12--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  4.27MB/s    in 0.2s    

2025-07-29 14:11:12 (4.27 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [ ]:
code = '''import pandas as pd
import random
import time
import os
from surprise import Dataset, Reader, SVD

# ✅ Հայկական ֆիլմերի ստատիկ ցուցակ
ARMENIAN_MOVIES = [
    (100001, "Երկունք (1991)", "Drama"),
    (100002, "Հին օրերի երգը (1982)", "Drama|History"),
    (100003, "Նռան գույնը (1984)", "Art|Drama"),
    (100004, "Մենք ենք, մեր սարերը (1969)", "Drama|Comedy"),
    (100005, "Տղամարդիկ (1972)", "Comedy"),
    (100006, "Կտոր մը երկինք (1980)", "Drama|Romance"),
    (100007, "Սարոյան եղբայրներ (2008)", "Documentary"),
    (100008, "Աղբյուրի մոտ (1970)", "Drama"),
    (100009, "Գարեգին Նժդեհ (2013)", "War"),
    (100010, "Եվա (2017)", "Drama"),
    (100011, "Հարսնացուն հյուսիսից (1975)", "Comedy|Romance"),
    (100012, "Մեր սրտի տնակը (2014)", "Drama"),
    (100013, "Եռանկունի (1967)", "Drama"),
    (100014, "Մայրիկ (1991)", "Drama"),
    (100015, "Խոստումը (2016)", "Drama")
]

def load_datasets(movielens_path="ml-latest-small"):
    """
    Բեռնում է Movielens և հայկական ֆիլմերի տվյալները։
    Վերադարձնում է՝ all_movies_df, all_ratings_df
    """
    armenian_movies_df = pd.DataFrame(ARMENIAN_MOVIES, columns=["movieId", "title", "genres"])

    # Արհեստական գնահատականներ 10 օգտատիրոջ համար
    user_ids = list(range(1001, 1011))
    ratings_data = []
    for user_id in user_ids:
        rated_movies = random.sample(armenian_movies_df["movieId"].tolist(), k=random.randint(4, 8))
        for movie_id in rated_movies:
            rating = round(random.uniform(3.0, 5.0), 1)
            timestamp = int(time.time())
            ratings_data.append((user_id, movie_id, rating, timestamp))
    armenian_ratings_df = pd.DataFrame(ratings_data, columns=["userId", "movieId", "rating", "timestamp"])

    # Movielens-ի բազայից ընթերցում
    movies = pd.read_csv(os.path.join(movielens_path, "movies.csv"))
    ratings = pd.read_csv(os.path.join(movielens_path, "ratings.csv"))

    # Միավորում ենք հայկական և Movielens տվյալները
    all_movies = pd.concat([movies, armenian_movies_df], ignore_index=True)
    all_ratings = pd.concat([ratings, armenian_ratings_df], ignore_index=True)

    return all_movies, all_ratings

def train_model(ratings_df):
    """
    Ստեղծում և մարզում է SVD մոդել՝ տրված գնահատականների վրա։
    """
    reader = Reader(rating_scale=(0.5, 5.0))
    data = Dataset.load_from_df(ratings_df[["userId", "movieId", "rating"]], reader)
    trainset = data.build_full_trainset()

    model = SVD()
    model.fit(trainset)

    return model

def get_movie_dict(movies_df):
    """
    Վերադարձնում է {վերնագիր: movieId} բառարան։
    """
    return dict(zip(movies_df["title"], movies_df["movieId"]))

def add_new_movies(movies_df, user_inputs):
    """
    Ստուգում է արդյոք նոր ֆիլմեր կան օգտագործողի մուտքածներում, և եթե կան՝ ավելացնում է movies_df-ում։
    user_inputs = [(title, rating), ...]
    """
    movie_title_to_id = get_movie_dict(movies_df)
    new_entries = []
    new_movie_id = movies_df["movieId"].max() + 1

    for title, _ in user_inputs:
        if title not in movie_title_to_id:
            genre = input(f"Ֆիլմը '{title}' չկա բազայում։ Գրեք ժանրը (օր. 'Drama, Comedy'): ")
            new_entries.append({
                "movieId": new_movie_id,
                "title": title,
                "genres": genre.strip()
            })
            movie_title_to_id[title] = new_movie_id
            new_movie_id += 1

    if new_entries:
        movies_df = pd.concat([movies_df, pd.DataFrame(new_entries)], ignore_index=True)

    return movies_df, movie_title_to_id

def prepare_new_ratings(user_inputs, movie_title_to_id, user_id=9999):
    """
    Վերափոխում է [(title, rating)] → [(userId, movieId, rating)]
    """
    new_ratings = []
    for title, rating in user_inputs:
        movie_id = movie_title_to_id[title]
        new_ratings.append((user_id, movie_id, rating))
    return pd.DataFrame(new_ratings, columns=["userId", "movieId", "rating"])

def get_recommendations(model, movies_df, watched_movie_ids, user_id=9999, top_n=10):
    """
    Վերադարձնում է լավագույն առաջարկությունները։
    """
    recommendations = []
    for movie_id in movies_df["movieId"].unique():
        if movie_id not in watched_movie_ids:
            pred = model.predict(user_id, movie_id)
            title = movies_df[movies_df["movieId"] == movie_id]["title"].values[0]
            recommendations.append((title, pred.est))
    recommendations.sort(key=lambda x: x[1], reverse=True)
    return recommendations[:top_n]
'''

with open("recommender.py", "w", encoding="utf-8") as f:
    f.write(code)

print("✅ recommender.py ֆայլը հաջողությամբ ստեղծվեց։")


✅ recommender.py ֆայլը հաջողությամբ ստեղծվեց։


In [ ]:
import recommender

movies_df, ratings_df = recommender.load_datasets()
print("Տվյալների բեռնման օրինակ՝")
print(movies_df.sample(5))
import recommender

movies_df, ratings_df = recommender.load_datasets()
print("Տվյալների բեռնման օրինակ՝")
print(movies_df.sample(5))


Տվյալների բեռնման օրինակ՝
      movieId                    title  \
853      1124    On Golden Pond (1981)   
7649    88235        Guard, The (2011)   
7189    72424          Derailed (2002)   
3477     4744  Jeepers Creepers (2001)   
6358    49530     Blood Diamond (2006)   

                                         genres  
853                                       Drama  
7649                               Comedy|Crime  
7189                            Action|Thriller  
3477                                     Horror  
6358  Action|Adventure|Crime|Drama|Thriller|War  
Տվյալների բեռնման օրինակ՝
      movieId                                      title  \
6093    42013                         Ringer, The (2005)   
2257     2995               House on Haunted Hill (1999)   
2216     2944                    Dirty Dozen, The (1967)   
4090     5847                             Ragtime (1981)   
8053    98809  Hobbit: An Unexpected Journey, The (2012)   

                      genres  
609

In [ ]:
user_inputs = []
while True:
    entry = input("> ")
    if entry.strip() == "":
        break
    try:
        title, rating = entry.rsplit(",", 1)
        title = title.strip()
        rating = float(rating.strip())
        user_inputs.append((title, rating))
    except Exception as e:
        print("Սխալ մուտք. Օգտագործեք `Վերնագիր, գնահատական` ձևաչափը։")

# Եթե կա նոր ֆիլմ, ավելացրու բազային
movies_df, movie_title_to_id = recommender.add_new_movies(movies_df, user_inputs)

# Փոխակերպիր օգտագործողի մուտքերը userId, movieId, rating տիպի DataFrame
new_ratings_df = recommender.prepare_new_ratings(user_inputs, movie_title_to_id, user_id=9999)

# Միացրու ամբողջ գնահատականների DataFrame-ին
full_ratings = pd.concat([ratings_df, new_ratings_df], ignore_index=True)

# Սովորեցրու մոդելը ամբողջ տվյալների վրա
model = recommender.train_model(full_ratings)

# Հաշվիր դիտած ֆիլմերի movieId-ները
watched_ids = [movie_title_to_id[title] for title, _ in user_inputs]

# Ստացիր առաջարկությունները
recommendations = recommender.get_recommendations(model, movies_df, watched_ids)

print("🎬 Առաջարկվող ֆիլմեր՝")
for i, (title, score) in enumerate(recommendations, 1):
    print(f"{i}. {title} ({round(score, 2)})")


> Me Before you, 5
> intersteller, 5
> 
Ֆիլմը 'Me Before you' չկա բազայում։ Գրեք ժանրը (օր. 'Drama, Comedy'): Drama
Ֆիլմը 'intersteller' չկա բազայում։ Գրեք ժանրը (օր. 'Drama, Comedy'): Sience fiction
🎬 Առաջարկվող ֆիլմեր՝
1. Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964) (4.83)
2. Matrix, The (1999) (4.78)
3. Godfather, The (1972) (4.72)
4. One Flew Over the Cuckoo's Nest (1975) (4.7)
5. Shawshank Redemption, The (1994) (4.69)
6. Lawrence of Arabia (1962) (4.67)
7. Rear Window (1954) (4.67)
8. Godfather: Part II, The (1974) (4.67)
9. Philadelphia Story, The (1940) (4.65)
10. Usual Suspects, The (1995) (4.64)
